# Data Wrangling

##### Making our scraped data easier to work with.

In [1]:
import pandas as pd
import numpy as np
import pickle

from pandas.io.json import json_normalize

def explode(df, lst_cols, fill_value=''):
    # make sure `lst_cols` is a list
    if lst_cols and not isinstance(lst_cols, list):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)

    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()

    if (lens > 0).all():
        # ALL lists in cells aren't empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, lens)
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .loc[:, df.columns]
    else:
        # at least one list in cells is empty
        return pd.DataFrame({
            col:np.repeat(df[col].values, lens)
            for col in idx_cols
        }).assign(**{col:np.concatenate(df[col].values) for col in lst_cols}) \
          .append(df.loc[lens==0, idx_cols]).fillna(fill_value) \
          .loc[:, df.columns]

Load data from local file containing our scraped data. 

In [2]:
crunchy = pickle.load(open("crunchy.pkl", "rb"))

In [3]:
df = pd.DataFrame(crunchy)

df.head()

,home_info,name,reviews
0,{'similar': ['BORUTO: NARUTO NEXT GENERATIONS'...,/07-ghost,[[{'useful': ['\n 43\n out of\n 56\n ...
1,{'similar': ['BORUTO: NARUTO NEXT GENERATIONS'...,/100-teacher-pascal,[[{'useful': ['\n 7\n out of\n 8\n ...
2,"{'similar': ['Asura Cryin'', 'BORUTO: NARUTO N...",/11eyes,[[{'useful': ['\n 48\n out of\n 59\n ...
3,"{'similar': ['MY love STORY!!', 'Tsukigakirei...",/5-centimeters-per-second,[[{'useful': ['\n 20\n out of\n 22\n ...
4,"{'similar': ['JOKER GAME', 'EVIL OR LIVE', 'Cl...",/91-days,[[{'useful': ['\n 26\n out of\n 26\n ...


In [4]:
home_info = df["home_info"].apply(pd.Series).rename(index=str, columns={0: "rating"})

home_info.head()

,similar,desc,rating,tags,num_eps
0,"[BORUTO: NARUTO NEXT GENERATIONS, One Piece, S...",\n Teased unmercifully for his past...,"{'agg_review': '4.5', 1: '(9)', 2: '(13)', 3: ...","[Yomiuri Telecasting Corporation (YTV), 07 gho...",25
1,"[BORUTO: NARUTO NEXT GENERATIONS, One Piece, S...",\n Pascal-sensei has been the teach...,"{'agg_review': '3.5', 1: '(1)', 2: '(0)', 3: '...",[ShoPro],36
2,"[Asura Cryin', BORUTO: NARUTO NEXT GENERATIONS...",\n Satsuki Kakeru lived an ordinary l...,"{'agg_review': '4.5', 1: '(42)', 2: '(43)', 3:...","[Marvelous AQL Inc., 11eyes, 11 eyes, tsumi to...",12
3,"[MY love STORY!!, Tsukigakirei, Hourou Musuko...",\n Beginning with the lyrical image...,"{'agg_review': '5', 1: '(16)', 2: '(7)', 3: '(...","[CoMixWave Films, drama, romance, beautiful, m...",2
4,"[JOKER GAME, EVIL OR LIVE, Classroom of the El...","\n During Prohibition, the law held...","{'agg_review': '4.5', 1: '(0)', 2: '(1)', 3: '...","[Shochiku, drama, historical, seinen]",27


In [5]:
ratings = home_info["rating"].apply(pd.Series)

ratings.head()

,agg_review,1,2,3,4,5
0,4.5,(9),(13),(26),(179),(1169)
1,3.5,(1),(0),(0),(1),(1)
2,4.5,(42),(43),(112),(375),(1566)
3,5,(16),(7),(27),(96),(1366)
4,4.5,(0),(1),(11),(16),(122)


In [6]:
home_to_join = home_info.join(ratings).drop("rating",axis = 1)

home_to_join.head()

,similar,desc,tags,num_eps,agg_review,1,2,3,4,5
0,"[BORUTO: NARUTO NEXT GENERATIONS, One Piece, S...",\n Teased unmercifully for his past...,"[Yomiuri Telecasting Corporation (YTV), 07 gho...",25,4.5,(9),(13),(26),(179),(1169)
1,"[BORUTO: NARUTO NEXT GENERATIONS, One Piece, S...",\n Pascal-sensei has been the teach...,[ShoPro],36,3.5,(1),(0),(0),(1),(1)
2,"[Asura Cryin', BORUTO: NARUTO NEXT GENERATIONS...",\n Satsuki Kakeru lived an ordinary l...,"[Marvelous AQL Inc., 11eyes, 11 eyes, tsumi to...",12,4.5,(42),(43),(112),(375),(1566)
3,"[MY love STORY!!, Tsukigakirei, Hourou Musuko...",\n Beginning with the lyrical image...,"[CoMixWave Films, drama, romance, beautiful, m...",2,5,(16),(7),(27),(96),(1366)
4,"[JOKER GAME, EVIL OR LIVE, Classroom of the El...","\n During Prohibition, the law held...","[Shochiku, drama, historical, seinen]",27,4.5,(0),(1),(11),(16),(122)


In [7]:
df.index = list(df.index)
home_to_join.index = home_to_join.index.astype("int64")

home_main = df.join(home_to_join).drop("home_info", axis = 1)

for i in range(1,6):
    home_main[i] = home_main[i].str.replace("(","").str.replace(")","")

home_main["name"] = home_main["name"].apply(lambda x : x[1:])

home_main["tags"] = home_main["tags"].apply(lambda x: "::".join(x))

home_main["similar"] = home_main["similar"].apply(lambda x: "::".join(x))

home_main.head()

,name,reviews,similar,desc,tags,num_eps,agg_review,1,2,3,4,5
0,07-ghost,[[{'useful': ['\n 43\n out of\n 56\n ...,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169
1,100-teacher-pascal,[[{'useful': ['\n 7\n out of\n 8\n ...,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Pascal-sensei has been the teach...,ShoPro,36,3.5,1,0,0,1,1
2,11eyes,[[{'useful': ['\n 48\n out of\n 59\n ...,Asura Cryin'::BORUTO: NARUTO NEXT GENERATIONS:...,\n Satsuki Kakeru lived an ordinary l...,Marvelous AQL Inc.::11eyes::11 eyes::tsumi to ...,12,4.5,42,43,112,375,1566
3,5-centimeters-per-second,[[{'useful': ['\n 20\n out of\n 22\n ...,MY love STORY!!::Tsukigakirei::Hourou Musuko ...,\n Beginning with the lyrical image...,CoMixWave Films::drama::romance::beautiful::ma...,2,5,16,7,27,96,1366
4,91-days,[[{'useful': ['\n 26\n out of\n 26\n ...,JOKER GAME::EVIL OR LIVE::Classroom of the Eli...,"\n During Prohibition, the law held...",Shochiku::drama::historical::seinen,27,4.5,0,1,11,16,122


In [8]:
home_main["reviews"] = (home_main["reviews"].
                 apply(lambda x : [item for sublist in x for item in sublist]))


home_main["reviews"] = home_main["reviews"].apply(json_normalize)

home_main["reviews"].head()

0                          datetime rating  \
0   \...
1                         datetime rating  \
0  \n ...
2                           datetime     rating  \
...
3                          datetime     rating  \
0...
4                           datetime rating  \
0   ...
Name: reviews, dtype: object

In [9]:
home_main.apply(lambda x: x["reviews"].insert(0,"show",x["name"],True), axis = 1)
print("insertion is done")

home_main["reviews"].head()

insertion is done


0            show                    datetime ratin...
1                     show                    datet...
2           show                    datetime     ra...
3                            show                  ...
4            show                    datetime ratin...
Name: reviews, dtype: object

In [10]:
reviews = pd.concat(home_main["reviews"].values,
                    sort = False)

reviews.head()

,show,datetime,rating,review,summary,useful
0,07-ghost,"\n Sep 19, 2014\n",2,\n I have NO clue why this show i...,The few good moments aren't worth the boring s...,[\n 43\n out of\n 56\n people fou...
1,07-ghost,"\n Jul 27, 2013\n",5,\n Synopsis: \r\nMain character ...,I demand season two pronto!,[\n 20\n out of\n 22\n people fou...
2,07-ghost,"\n Aug 3, 2012\n",4,\n I really liked this show. A de...,A good show but.....,[\n 9\n out of\n 11\n people foun...
3,07-ghost,"\n Feb 22, 2015\n",2,\n The first episode throws off t...,Bad pacin... ...g ...,[\n 10\n out of\n 14\n people fou...
4,07-ghost,"\n Jan 27, 2013\n",3,\n Well... I love the opening and...,Just an average rating,[\n 6\n out of\n 6\n people found...


In [11]:
home_main_review_raw = pd.merge(home_main,reviews,
                                left_on="name",
                                right_on="show")

home_main_review_raw.head()

,name,reviews,similar,desc,tags,num_eps,agg_review,1,2,3,4,5,show,datetime,rating,review,summary,useful
0,07-ghost,show datetime ratin...,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,07-ghost,"\n Sep 19, 2014\n",2,\n I have NO clue why this show i...,The few good moments aren't worth the boring s...,[\n 43\n out of\n 56\n people fou...
1,07-ghost,show datetime ratin...,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,07-ghost,"\n Jul 27, 2013\n",5,\n Synopsis: \r\nMain character ...,I demand season two pronto!,[\n 20\n out of\n 22\n people fou...
2,07-ghost,show datetime ratin...,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,07-ghost,"\n Aug 3, 2012\n",4,\n I really liked this show. A de...,A good show but.....,[\n 9\n out of\n 11\n people foun...
3,07-ghost,show datetime ratin...,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,07-ghost,"\n Feb 22, 2015\n",2,\n The first episode throws off t...,Bad pacin... ...g ...,[\n 10\n out of\n 14\n people fou...
4,07-ghost,show datetime ratin...,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,07-ghost,"\n Jan 27, 2013\n",3,\n Well... I love the opening and...,Just an average rating,[\n 6\n out of\n 6\n people found...


In [12]:
home_main_review_raw = (explode(home_main_review_raw,"useful").
                        drop(["reviews","show"],
                             axis = 1))

home_main_review_raw.head()

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


,name,similar,desc,tags,num_eps,agg_review,1,2,3,4,5,datetime,rating,review,summary,useful
0,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Sep 19, 2014\n",2,\n I have NO clue why this show i...,The few good moments aren't worth the boring s...,\n 43\n out of\n 56\n people foun...
1,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Jul 27, 2013\n",5,\n Synopsis: \r\nMain character ...,I demand season two pronto!,\n 20\n out of\n 22\n people foun...
2,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Aug 3, 2012\n",4,\n I really liked this show. A de...,A good show but.....,\n 9\n out of\n 11\n people found...
3,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Feb 22, 2015\n",2,\n The first episode throws off t...,Bad pacin... ...g ...,\n 10\n out of\n 14\n people foun...
4,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Jan 27, 2013\n",3,\n Well... I love the opening and...,Just an average rating,\n 6\n out of\n 6\n people found ...


In [13]:
home_main_review_raw.head()

,name,similar,desc,tags,num_eps,agg_review,1,2,3,4,5,datetime,rating,review,summary,useful
0,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Sep 19, 2014\n",2,\n I have NO clue why this show i...,The few good moments aren't worth the boring s...,\n 43\n out of\n 56\n people foun...
1,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Jul 27, 2013\n",5,\n Synopsis: \r\nMain character ...,I demand season two pronto!,\n 20\n out of\n 22\n people foun...
2,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Aug 3, 2012\n",4,\n I really liked this show. A de...,A good show but.....,\n 9\n out of\n 11\n people found...
3,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Feb 22, 2015\n",2,\n The first episode throws off t...,Bad pacin... ...g ...,\n 10\n out of\n 14\n people foun...
4,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Jan 27, 2013\n",3,\n Well... I love the opening and...,Just an average rating,\n 6\n out of\n 6\n people found ...


## 3 Main Files

In [14]:
home_info = home_main.drop("reviews",axis = 1)

home_info.to_csv("crunchy_home.csv", index = False)

In [17]:
home_info.head()

,name,similar,desc,tags,num_eps,agg_review,1,2,3,4,5
0,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169
1,100-teacher-pascal,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Pascal-sensei has been the teach...,ShoPro,36,3.5,1,0,0,1,1
2,11eyes,Asura Cryin'::BORUTO: NARUTO NEXT GENERATIONS:...,\n Satsuki Kakeru lived an ordinary l...,Marvelous AQL Inc.::11eyes::11 eyes::tsumi to ...,12,4.5,42,43,112,375,1566
3,5-centimeters-per-second,MY love STORY!!::Tsukigakirei::Hourou Musuko ...,\n Beginning with the lyrical image...,CoMixWave Films::drama::romance::beautiful::ma...,2,5,16,7,27,96,1366
4,91-days,JOKER GAME::EVIL OR LIVE::Classroom of the Eli...,"\n During Prohibition, the law held...",Shochiku::drama::historical::seinen,27,4.5,0,1,11,16,122


In [15]:
review_info = explode(reviews,"useful")

review_info.to_csv("crunchy_review.csv", index= False)

In [18]:
review_info.head()

,show,datetime,rating,review,summary,useful
0,07-ghost,"\n Sep 19, 2014\n",2,\n I have NO clue why this show i...,The few good moments aren't worth the boring s...,\n 43\n out of\n 56\n people foun...
1,07-ghost,"\n Jul 27, 2013\n",5,\n Synopsis: \r\nMain character ...,I demand season two pronto!,\n 20\n out of\n 22\n people foun...
2,07-ghost,"\n Aug 3, 2012\n",4,\n I really liked this show. A de...,A good show but.....,\n 9\n out of\n 11\n people found...
3,07-ghost,"\n Feb 22, 2015\n",2,\n The first episode throws off t...,Bad pacin... ...g ...,\n 10\n out of\n 14\n people foun...
4,07-ghost,"\n Jan 27, 2013\n",3,\n Well... I love the opening and...,Just an average rating,\n 6\n out of\n 6\n people found ...


In [16]:
home_main_review_raw.to_csv("crunchy_main.csv", index = False)

In [19]:
home_main_review_raw.head()

,name,similar,desc,tags,num_eps,agg_review,1,2,3,4,5,datetime,rating,review,summary,useful
0,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Sep 19, 2014\n",2,\n I have NO clue why this show i...,The few good moments aren't worth the boring s...,\n 43\n out of\n 56\n people foun...
1,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Jul 27, 2013\n",5,\n Synopsis: \r\nMain character ...,I demand season two pronto!,\n 20\n out of\n 22\n people foun...
2,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Aug 3, 2012\n",4,\n I really liked this show. A de...,A good show but.....,\n 9\n out of\n 11\n people found...
3,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Feb 22, 2015\n",2,\n The first episode throws off t...,Bad pacin... ...g ...,\n 10\n out of\n 14\n people foun...
4,07-ghost,BORUTO: NARUTO NEXT GENERATIONS::One Piece::Sw...,\n Teased unmercifully for his past...,Yomiuri Telecasting Corporation (YTV)::07 ghos...,25,4.5,9,13,26,179,1169,"\n Jan 27, 2013\n",3,\n Well... I love the opening and...,Just an average rating,\n 6\n out of\n 6\n people found ...
